In [36]:
import torch
from copy import deepcopy

sd = torch.load('/home/yujiepan/work2/jpqd-vit/LOGS/ww42/1012.689a-vit-jpqnd-wt0wr0.060-prune2to7f8-epo45lr5e-5wd1e-6_2card/model_43.pth')['model']

In [2]:
keys = []
new_sd = deepcopy(sd)
for i in range(12):
    for m in ["q", "k", "v", "out"]:
        keys.append(f"nncf_module.encoder.layers.encoder_layer_{i}.self_attention.{m}_proj")
    for m in [0, 3]:
        keys.append(f"nncf_module.encoder.layers.encoder_layer_{i}.mlp.{m}")

for key in keys:
    for d in ["weight", "bias"]:
        importance = f"{key}.pre_ops.0.op._{d}_importance"
        # print('patching', importance)
        new_sd[importance] = torch.ones_like(sd[importance]) * 100.0
        new_sd[f"{key}.{d}"] = sd[f"{key}.{d}"] * sd[f"{key}.pre_ops.0.op.{d}_ctx._binary_mask"]
        new_sd[f"{key}.pre_ops.0.op.{d}_ctx._binary_mask"] = torch.ones_like(sd[f"{key}.pre_ops.0.op.{d}_ctx._binary_mask"])

In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
plt.rcParams["savefig.bbox"] = 'tight'
def show(tensor):
    imgs = tensor
    for i in range(3 - len(tensor.shape)):
        imgs = imgs.unsqueeze(0)
    imgs = torchvision.utils.make_grid(imgs)
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [15]:
keys = []

def need_to_preserve_head(layer_id, head_id, new_sd):
    for m in ["q", "k", "v"]:
        weight = new_sd[f"nncf_module.encoder.layers.encoder_layer_{layer_id}.self_attention.{m}_proj.weight"]
        if (weight[head_id*64 : (head_id+1)*64, :].abs() > 1e-10).float().sum() > 0:
            # print(m)
            return True
        bias = new_sd[f"nncf_module.encoder.layers.encoder_layer_{layer_id}.self_attention.{m}_proj.bias"]
        bias = bias.reshape(-1)
        if (bias[head_id*64 : (head_id+1)*64].abs() > 1e-10).float().sum() > 0:
            # print(m, 'bias')
            return True
    m = 'out'
    weight = new_sd[f"nncf_module.encoder.layers.encoder_layer_{layer_id}.self_attention.{m}_proj.weight"]
    if (weight[:, head_id*64 : (head_id+1)*64].abs() > 1e-10).float().sum() > 0:
        return True
    return False

def need_to_preserve_ffn(layer_id, ffn_id, new_sd):
    weight = new_sd[f'nncf_module.encoder.layers.encoder_layer_{layer_id}.mlp.0.weight']
    if (weight[ffn_id, :].abs() > 1e-10).float().sum() > 0:
        return True
    weight = new_sd[f'nncf_module.encoder.layers.encoder_layer_{layer_id}.mlp.3.weight']
    if (weight[:, ffn_id].abs() > 1e-10).float().sum() > 0:
        return True
    bias = new_sd[f'nncf_module.encoder.layers.encoder_layer_{layer_id}.mlp.0.bias']
    bias = bias.reshape(-1)
    if (bias[ffn_id].abs() > 1e-10).float().sum() > 0:
        return True
    return False

for layer_id in range(12):
    preserved_heads = []
    preserved_ffns = []
    for head_id in range(12):
        if need_to_preserve_head(layer_id, head_id, new_sd):
            preserved_heads.append(head_id)
    for ffn_id in range(768 * 4):
        if need_to_preserve_ffn(layer_id, ffn_id, new_sd):
            preserved_ffns.append(ffn_id)
    print(layer_id, preserved_heads, len(preserved_ffns))


0 [0, 1, 7, 9, 11] 826
1 [2, 4, 8] 1508
2 [5, 9, 10, 11] 2100
3 [0, 2, 3, 5, 6, 7, 9, 10] 2143
4 [0, 1, 3, 5, 6, 7, 8, 9, 10, 11] 2157
5 [0, 3, 4, 5, 7, 8, 9, 10, 11] 2167
6 [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11] 2219
7 [0, 1, 2, 3, 5, 8, 9, 10, 11] 2374
8 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11] 2147
9 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 2329
10 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 2857
11 [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11] 1594


In [ ]:

i = 1
import torchvision
m = 0

x = sd[f'nncf_module.encoder.layers.encoder_layer_{i}.mlp.{m}.weight']
show((x.abs() > 2e-10).float())
print(x.shape)

show((sd[f'nncf_module.encoder.layers.encoder_layer_{i}.mlp.{m}.pre_ops.0.op.bias_ctx._binary_mask']).repeat(800, 1))
y = sd[f'nncf_module.encoder.layers.encoder_layer_{i}.mlp.{m}.pre_ops.0.op.bias_ctx._binary_mask']
x = sd[f'nncf_module.encoder.layers.encoder_layer_{i}.mlp.{m}.pre_ops.0.op.weight_ctx._binary_mask']
x.shape, y.shape
show(x)
torch.testing.assert_allclose(x.mean(dim=1), y)

In [14]:
from pathlib import Path
import pandas as pd
import json

items = []
for json_path in Path('/home/yujiepan/work2/jpqd-vit/LOGS/2023-re-collect-summary-v2').glob('*/summary.json'):
    with open(json_path, 'r') as f:
        result = json.load(f)
        model = result['model_path']
        model = model[model.index('ww42'):]
        model = model[:14] + model[-7:]
        items.append(dict(
            model=model, acc=result['top1'], sparsity=result['linear_sparsity'],
            drop=100*(result['top1'] / 81.068 - 1),
        ))

df = pd.DataFrame(items)
df = df.sort_values(['model', 'sparsity'])
print(df)
df

                    model     acc  sparsity      drop
14  ww42/1008.713e_40.pth  80.654  0.255133 -0.510682
11  ww42/1008.713e_41.pth  80.718  0.255133 -0.431736
13  ww42/1008.713e_42.pth  80.640  0.255133 -0.527952
12  ww42/1008.713e_43.pth  80.690  0.255133 -0.466275
8   ww42/1008.713e_44.pth  80.664  0.255133 -0.498347
6   ww42/1011.6586_40.pth  80.470  0.303244 -0.737652
9   ww42/1011.6586_41.pth  80.332  0.303244 -0.907880
1   ww42/1011.6586_42.pth  80.368  0.303244 -0.863473
5   ww42/1011.6586_43.pth  80.370  0.303244 -0.861006
4   ww42/1011.6586_44.pth  80.422  0.303244 -0.796862
2   ww42/1012.689a_40.pth  80.214  0.317545 -1.053437
3   ww42/1012.689a_41.pth  80.302  0.317545 -0.944886
7   ww42/1012.689a_42.pth  80.248  0.317545 -1.011497
10  ww42/1012.689a_43.pth  80.320  0.317545 -0.922682
0   ww42/1012.689a_44.pth  80.216  0.317545 -1.050970


,model,acc,sparsity,drop
14,ww42/1008.713e_40.pth,80.654,0.255133,-0.510682
11,ww42/1008.713e_41.pth,80.718,0.255133,-0.431736
13,ww42/1008.713e_42.pth,80.640,0.255133,-0.527952
12,ww42/1008.713e_43.pth,80.690,0.255133,-0.466275
8,ww42/1008.713e_44.pth,80.664,0.255133,-0.498347
6,ww42/1011.6586_40.pth,80.470,0.303244,-0.737652
9,ww42/1011.6586_41.pth,80.332,0.303244,-0.907880
1,ww42/1011.6586_42.pth,80.368,0.303244,-0.863473
5,ww42/1011.6586_43.pth,80.370,0.303244,-0.861006
4,ww42/1011.6586_44.pth,80.422,0.303244,-0.796862


# Manual crop"

In [4]:
import torch
import nncf
import torchvision
import shutil
from nncf import NNCFConfig
from nncf.torch import register_default_init_args
from nncf.torch import create_compressed_model
import jstyleson

In [5]:
model = torchvision.models.vit_b_16(weights=None, num_classes=1000)
with open("/nvme2/yujiepan/workspace/jpqd-vit/LOGS/ww42/1011.6586-vit-jpqnd-wt0wr0.055-prune2to7f8-epo45lr5e-5wd1e-6_2card/vit_b16_jpqnd_2to7f8_wt0wr0.055_2card.ft.json", 'r') as f:
    nncf_config = jstyleson.load(f)

nncf_config["log_dir"] = '/tmp'
override_qcfg_init = dict(
    range=dict(num_init_samples=0), batchnorm_adaptation=dict(num_bn_adaptation_samples=0)
)
if isinstance(nncf_config["compression"], list):
    for algo in nncf_config["compression"]:
        if algo["algorithm"] == "quantization":
            algo["initializer"].update(override_qcfg_init)
        if algo["algorithm"] == 'movement_sparsity':
            algo['params'] =  {
                "warmup_start_epoch":  1,
                "warmup_end_epoch":    4,
                "importance_regularization_factor":  0.01,
                "enable_structured_masking":  False
            }
            algo['sparse_structure_by_scopes'] = [
                {"mode":  "block",   "sparse_factors": [32, 32], "target_scopes": "{re}.*MultiHeadAttention*"},
                {"mode":  "per_dim", "axis":  0,                 "target_scopes": "{re}.*MLPBlock.mlp./NNCFLinear.0"},
                {"mode":  "per_dim", "axis":  1,                 "target_scopes": "{re}.*MLPBlock.mlp./NNCFLinear.3"},
            ]
elif nncf_config["compression"]["algorithm"] == "quantization":
    nncf_config["compression"]["initializer"].update(override_qcfg_init)


class RandDataset(torch.utils.data.Dataset):
    def __init__(self) -> None:
        super().__init__()
    
    def __len__(self):
        return 4
    
    def __getitem__(self, i):
        return torch.randn((3, 224, 224)), 0

fake_data_loader = torch.utils.data.DataLoader(RandDataset(), batch_size=4)

nncf_config = NNCFConfig.from_dict(nncf_config)
nncf_config = register_default_init_args(
nncf_config=nncf_config, train_loader=fake_data_loader
)  # TODO: distributed_callbacks and execution_parameters
print(nncf_config)

compression_ctrl, model = create_compressed_model(model, nncf_config)

{'input_info': [{'sample_size': [1, 3, 224, 224]}], 'compression': [{'algorithm': 'movement_sparsity', 'params': {'warmup_start_epoch': 1, 'warmup_end_epoch': 4, 'importance_regularization_factor': 0.01, 'enable_structured_masking': False}, 'sparse_structure_by_scopes': [{'mode': 'block', 'sparse_factors': [32, 32], 'target_scopes': '{re}.*MultiHeadAttention*'}, {'mode': 'per_dim', 'axis': 0, 'target_scopes': '{re}.*MLPBlock.mlp./NNCFLinear.0'}, {'mode': 'per_dim', 'axis': 1, 'target_scopes': '{re}.*MLPBlock.mlp./NNCFLinear.3'}], 'ignored_scopes': ['{re}.*VisionTransformer/NNCFConv2d.conv_proj.*', '{re}.*VisionTransformer/Sequential.heads.*', '{re}.*class_token_layer.*', '{re}.*pos_embedding_layer.*']}, {'algorithm': 'quantization', 'initializer': {'range': {'num_init_samples': 0}, 'batchnorm_adaptation': {'num_bn_adaptation_samples': 0}}, 'activations': {'mode': 'symmetric'}, 'weights': {'mode': 'symmetric', 'signed': True, 'per_channel': False}}], 'log_dir': '/tmp'}
INFO:nncf:Ignored

In [6]:
from state_dict_patch import resolve_structured_mask

new_sd, preserved_by_layer = resolve_structured_mask(torch.load('/nvme2/yujiepan/workspace/jpqd-vit/LOGS/ww42/1011.6586-vit-jpqnd-wt0wr0.055-prune2to7f8-epo45lr5e-5wd1e-6_2card/model_40.pth')['model'])


In [33]:
nnnew_sd = {}
for k, v in new_sd.items():
    if k.startswith('nncf_module.'):
        k = k.replace('nncf_module.', '')
    if k.startswith('external_quantizers'):
        k = f'_nncf.{k}'
    if k.endswith('._weight_importance'):
        k = k.replace('._weight_importance', '.weight_importance')
    if k.endswith('._bias_importance'):
        k = k.replace('._bias_importance', '.bias_importance')
    if '/LayerNorm[' in k:
        k = k.replace('/LayerNorm[', '/NNCFLayerNorm[')

    nnnew_sd[k] = v

In [38]:
model.load_state_dict(nnnew_sd)

<All keys matched successfully>